In [13]:
import numpy as np
import os
import cv2
import pyautogui
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *

In [14]:
def eye_aspect_ratio(eye):
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    ear = (A + B) / (2.0 * C)
    return ear


In [15]:
cascade = cv2.CascadeClassifier("D:/project/haarcascade_eye.xml")
video_capture = cv2.VideoCapture(0)

In [16]:
def normalize(x):
    minn, maxx = x.min(), x.max()
    return (x - minn) / (maxx - minn)

In [17]:
def scan(image_size=(32, 32)):
    
    boxes = cascade.detectMultiScale(gray, 1.3, 10)
    if len(boxes) == 2:
        eyes = []
        for box in boxes:
            x, y, w, h = box
            eye = frame[y:y + h, x:x + w]
            eye = cv2.resize(eye, image_size)
            eye = normalize(eye)
            eye = eye[10:-10, 5:-5]
            eyes.append(eye)
        return (np.hstack(eyes) * 255).astype(np.uint8)
    else:
        return None

In [18]:
width, height = 1919, 1079

In [19]:
root="D:/project/eye_pics/"
filepaths = os.listdir(root)
X, Y = [], []
for filepath in filepaths:
    x, y, _ = filepath.split(' ')
    x = float(x) / width
    y = float(y) / height
    X.append(cv2.imread(root + filepath))
    Y.append([x, y])
X = np.array(X) / 255.0
Y = np.array(Y)
print (X.shape, Y.shape)

(1236, 12, 44, 3) (1236, 2)


In [20]:
model = Sequential()
model.add(Conv2D(32, 3, 2, activation = 'relu', input_shape = (12, 44, 3)))
model.add(Conv2D(64, 2, 2, activation = 'relu'))
model.add(Flatten())
model.add(Dense(32, activation = 'relu'))
model.add(Dense(2, activation = 'sigmoid'))
model.compile(optimizer = "adam", loss = "mean_squared_error")
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 5, 21, 32)         896       
                                                                 
 conv2d_1 (Conv2D)           (None, 2, 10, 64)         8256      
                                                                 
 flatten (Flatten)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 32)                40992     
                                                                 
 dense_1 (Dense)             (None, 2)                 66        
                                                                 
Total params: 50,210
Trainable params: 50,210
Non-trainable params: 0
_________________________________________________________________


In [21]:
epochs = 200
for epoch in range(epochs):
    model.fit(X, Y, batch_size = 32)

39/39 [==============================] - 0s 7ms/step - loss: 6.2122e-04


39/39 [==============================] - 0s 7ms/step - loss: 4.2123e-04


In [22]:
import pyautogui
screen_width, screen_height = pyautogui.size()

In [23]:
import dlib


detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("D:/project/shape_predictor_68_face_landmarks.dat")

In [27]:
LEFT_EYE_POINTS = list(range(36, 42))
RIGHT_EYE_POINTS = list(range(42, 48))

# Define the EAR threshold for detecting eye blinks
EAR_THRESHOLD = 0.2

# Define the number of consecutive frames the EAR is below the threshold to detect a blink
CONSECUTIVE_FRAMES_THRESHOLD = 3

# Initialize the frame counters and the blink status
frame_counter = 0
left_eye_blinking = False
right_eye_blinking = False
cap = cv2.VideoCapture(0)

In [ ]:
import cv2
while True:
    _, frame = cap.read()
    if not _:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    eyes = scan()
    if not eyes is None:
        eyes = np.expand_dims(eyes / 255.0, axis = 0)
        x, y = model.predict(eyes)[0]
        pyautogui.moveTo(x * width, y * height)
    
    
    # Loop over the detected faces
    for face in faces:
        # Get the facial landmarks for the face region
        landmarks = predictor(gray, face)

        # Get the landmark points for the left and right eye regions
        left_eye_points = np.array([(landmarks.part(index).x, landmarks.part(index).y) for index in LEFT_EYE_POINTS])
        right_eye_points = np.array([(landmarks.part(index).x, landmarks.part(index).y) for index in RIGHT_EYE_POINTS])
        for eye in [left_eye_points, right_eye_points]:
            eye_center = np.mean(eye, axis=0).astype(int)
            eye_radius = int(np.linalg.norm(eye[0] - eye[3]) / 2)
            cv2.circle(frame, tuple(eye_center), eye_radius, (0, 255, 0), 2)
        # Calculate the EAR for each eye
        left_ear = eye_aspect_ratio(left_eye_points)
        right_ear = eye_aspect_ratio(right_eye_points)

        # Check if the EAR is below the threshold for the left eye
        if left_ear < EAR_THRESHOLD:
            if not left_eye_blinking:
                left_eye_blinking = True
                frame_counter = 0
            else:
                frame_counter += 1
                if frame_counter >= CONSECUTIVE_FRAMES_THRESHOLD:
                    pyautogui.click()
                    print("Left eye blinking")
        else:
            left_eye_blinking = False

        # Check if the EAR is below the threshold for the right eye
        if right_ear < EAR_THRESHOLD:
            if not right_eye_blinking:
                right_eye_blinking = True
                frame_counter = 0
            else:
                if frame_counter >= CONSECUTIVE_FRAMES_THRESHOLD:
                    pyautogui.rightClick()
                    print("right click")
        else:
            right_eye_blinking=False
    cv2.imshow('Eye Detection', frame)
    
    # Exit if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()   
        
        
        
        
        
        

1/1 [==============================] - 0s 21ms/step
Left eye blinking
1/1 [==============================] - 0s 46ms/step


1/1 [==============================] - 0s 23ms/step
Left eye blinking
right click
1/1 [==============================] - 0s 48ms/step


1/1 [==============================] - 0s 27ms/step
Left eye blinking
right click
Left eye blinking
right click
1/1 [==============================] - 0s 26ms/step
Left eye blinking
1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 0s 28ms/step
